In [9]:
#used to convert the pdf to an image
from pdf2image import convert_from_path
#exact text from images using pytesseract
import pytesseract 
import cv2
import numpy as np
#Image from Pillow makes the pic printable as a image and not only an array
from PIL import Image
import re

In [77]:
pages = convert_from_path(r'docs\patient_details\pd_1.pdf', poppler_path= r'C:\Users\nessa\poppler-22.04.0\Library\bin')

In [73]:
pages[0].show()
pages

[<PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1500x2000>,
 <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1992x2000>]

In [74]:
def preprocess_image(image):
    # image comes in and array comes out
    image_gray = cv2.cvtColor(np.array(image), cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(image_gray, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_LINEAR)
    # we need adaptive thresholding because there are many different shade colors in the image
    processed = cv2.adaptiveThreshold(
        resized, 255,
        # specify that it is the adaptive threst
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        # and that the pic gonna be white&black
        cv2.THRESH_BINARY,
        # block size of the black area
        61,
        # constant
        11
    )
    return processed

In [78]:
# i send an image, but i get back an array so we need pillow
img = preprocess_image(pages[0])
Image.fromarray(img).show()

In [79]:
# let's try to exact the text after the pre-processing
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
text = pytesseract.image_to_string(img, lang="eng")
print(text)

17/12/2020

Patient Medical Record

Patient Information Birth Date

Kathy Crawford May 6 1972

(737) 988-0851 Weight’

9264 Ash Dr 95

New York City, 10005 '

United States Height:
190

In Case of Emergency
ee J
Simeone Crawford 9266 Ash Dr
New York City, New York, 10005
Home phone United States
(990) 375-4621
Work phone
Genera! Medical History
nn ee
Chicken Pox (Varicella): Measies:
IMMUNE

IMMUNE
Have you had the Hepatitis B vaccination?

No

List any Medical Problems (asthma, seizures, headaches}:

Migraine



<h3> Extract name <h3>

In [51]:
# time to decode the text!
# starting with the name
import re 

pattern = "Patient Information(.*?)\(\d{3}\)"
matches = re.findall(pattern, text, flags = re.DOTALL)
matches

[' Birth Date\n\nKathy Crawford May 6 1972\n\n']

In [49]:
matches[0].strip()

'Birth Date\n\nKathy Crawford May 6 1972'

In [41]:
match = matches[0].replace("Birth Date", "").strip()
pattern = '((Jan|Feb|March|April|May|June|July|Aug|Sep|Oct|Nov|Dec)[ 0-9]+)'
date_matches = re.findall(pattern, match)
date = date_matches[0][0]
print(match, date)

Kathy Crawford May 6 1972 May 6 1972


In [48]:
match = match.replace(date,'').strip()
match

'Kathy Crawford'

In [61]:
def remove_noise_from_name(name):
    name = name.replace("Birth Date", "").strip()
    pattern = '((Jan|Feb|March|April|May|June|July|Aug|Sep|Oct|Nov|Dec)[ 0-9]+)'
    date_matches = re.findall(pattern, name)
    if date_matches:
        date =  date_matches[0][0]
        name = name.replace(date,'').strip()
    return name

In [62]:
name = ' Birth Date\n\nKathy Crawford May 6 1972\n\n'
print(remove_noise_from_name(name))

Kathy Crawford


<h3> Extract phone <h3>

In [66]:
pattern = 'Patient Information(.*?)(\(\d{3}\) \d{3}-\d{4})'
matches = re.findall(pattern, text, flags = re.DOTALL)
# 1st group is the 1st match, and 2nd group is the 2nd match
matches[0][1]

'(737) 988-0851'

<h3> Extract vaccine <h3>

In [69]:
pattern = 'Have you had the Hepatitis B vaccination\?(.*?)List'
matches = re.findall(pattern, text, flags = re.DOTALL)
matches[0].strip()

'No'

<h3> Extract medical problems <h3>

In [71]:
pattern = 'List any Medical Problems \(asthma, seizures, headaches}:(.*)'
matches = re.findall(pattern, text, flags = re.DOTALL)
matches[0].strip()

'Migraine'